In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import os
import utilys as ut
import datas
from LSN import Latent_Similarity_Network

In [2]:
print('Loading Data..')

df, inds, counts, slices, ordered = datas.get_data()
imgs = datas.load_imgs(df,100)
params_gen_epoch = inds, counts, slices, ordered, imgs

Loading Data..
0 images loaded..
1000 images loaded..
2000 images loaded..
3000 images loaded..
4000 images loaded..
5000 images loaded..
6000 images loaded..
7000 images loaded..
8000 images loaded..
9000 images loaded..
10000 images loaded..
11000 images loaded..
12000 images loaded..
13000 images loaded..
14000 images loaded..
15000 images loaded..
16000 images loaded..
17000 images loaded..
18000 images loaded..
19000 images loaded..


In [6]:
width = 100
batch_size = 16
learning_rate = 3e-6

n_inputs = width*width*3
conv_features = 7500
n_hnodes = 4000
latent_dim = 400
n_classes = 5

tf.reset_default_graph()

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

print('Initializing Weight Variables..')

W_init = tf.contrib.layers.xavier_initializer()
b_init = tf.constant_initializer(0)
with tf.variable_scope('Weights'):
    W_c1 = tf.get_variable('W_conv_1', shape=[5,5,3,6], initializer=W_init)
    b_c1 = tf.get_variable('b_conv_1', shape=[6])
    W_c2 = tf.get_variable('W_conv_2', shape=[5,5,6,12], initializer=W_init)
    b_c2 = tf.get_variable('b_conv_2', shape=[12])
    W_d1 = tf.get_variable('W_dense_1', shape=[conv_features,n_hnodes], initializer=W_init)
    b_d1 = tf.get_variable('b_dense_1', shape=[n_hnodes], initializer=b_init)
    W_d2 = tf.get_variable('W_dense_2', shape=[n_hnodes,latent_dim], initializer=W_init)
    b_d2 = tf.get_variable('b_dense_2', shape=[latent_dim], initializer=b_init)
thetas = [W_c1,b_c1,W_c2,b_c2,W_d1,b_d1,W_d2,b_d2]


print('Compiling Ocular Layers..')

X = tf.placeholder(tf.float32,[batch_size,None,width,width,3])
y = tf.placeholder(tf.float32,[batch_size,None,n_classes])

X_conv = tf.reshape(X,[-1,width,width,3])
h_c1 = conv2d(X_conv,W_c1)+b_c1
h_p1 = tf.nn.relu(max_pool(h_c1))
h_c2 = conv2d(h_p1,W_c2)+b_c2
h_p2 = tf.nn.relu(max_pool(h_c2))

X_fc = tf.reshape(h_p2,[-1,conv_features])
h_d1 = tf.nn.tanh(tf.matmul(X_fc,W_d1)+b_d1)
h_d2 = tf.nn.tanh(tf.matmul(h_d1,W_d2)+b_d2)
h_fc = tf.reshape(h_d2,[batch_size,-1,latent_dim])

y_tr = tf.transpose(y,perm=[1,0,2])
y_tm1 = tf.concat([tf.zeros([1,batch_size,n_classes]),y_tr[:-1,:]],0)
X_lsn = tf.transpose(h_fc,[1,0,2])



print('Compiling Latent Similarity Network..')

LSN0 = Latent_Similarity_Network(latent_dim,n_classes,batch_size,X_lsn,y_tr)
M, h = LSN0['M'], LSN0['h']



print('Compiling Optimizer..')

xentropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_tr,logits=h)
loss = tf.reduce_mean(xentropy)
optimizer = tf.train.AdamOptimizer(learning_rate)
grads, _ = tf.clip_by_global_norm(tf.gradients(loss,thetas),30)
train_step = optimizer.apply_gradients(zip(grads,thetas))
correct_prediction = tf.equal(tf.argmax(h, -1), tf.argmax(y_tr, -1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

tf.summary.scalar('loss',loss)
tf.summary.scalar('accuracy',accuracy)
tf.summary.scalar('memory_mean',tf.reduce_mean(M))
merged = tf.summary.merge_all()


print('TF Graph Successfully Compiled!')

Initializing Weight Variables..
Compiling Ocular Layers..
Compiling Latent Similarity Network..
Compiling Optimizer..
TF Graph Successfully Compiled!


In [7]:
savedir = './saver/LSN3'
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

writer = tf.summary.FileWriter(savedir+'/tb')
saver = tf.train.Saver(var_list=thetas)

saver.restore(sess, './saver/LSN3/2700/Model.ckpt')

INFO:tensorflow:Restoring parameters from ./saver/LSN3/2700/Model.ckpt


In [ ]:
n_epochs = 1400
load_at = 2700
save_every = 100

for epoch in range(n_epochs):
    # wrangling
    X_ep, y_ep = datas.gen_epoch(inds,counts,slices,ordered,imgs)
    n_batches = len(y_ep)//batch_size
    for batch in range(n_batches):
        X_batch = X_ep[batch*batch_size:(batch+1)*batch_size]
        y_batch = y_ep[batch*batch_size:(batch+1)*batch_size]
        length_batch = min([len(v) for v in y_batch])
        X_shuff, y_shuff = [], []
        for i in range(batch_size):
            X_episode, y_episode = ut.shuffle_xy(X_batch[i],y_batch[i])
            X_shuff.append(X_episode[:length_batch])
            y_shuff.append(y_episode[:length_batch])
        X_train, y_train = np.asarray(X_shuff)/127.5-1, np.asarray(y_shuff)
        feed_dict = {X:X_train,y:y_train}
        # run
        _, curr_loss, curr_accuracy, summary = sess.run((train_step,loss,accuracy,merged),feed_dict=feed_dict)
        print('Epoch %s (%s/%s) // Loss %.6f // Accuracy %.4f'%(epoch+1,batch+1,n_batches,curr_loss,curr_accuracy))
        # save
        writer.add_summary(summary,(load_at+epoch+1)*n_batches+(batch+1))
    if save_every>0 and (epoch+1)%save_every == 0:
        dir_iter = savedir+'/{}'.format(load_at+epoch+1)
        if not os.path.exists(dir_iter):
            os.makedirs(dir_iter)
        saver.save(sess,'{}/Model.ckpt'.format(dir_iter))

Epoch 1 (1/21) // Loss 1.072477 // Accuracy 0.8559
Epoch 1 (2/21) // Loss 1.091553 // Accuracy 0.8405
Epoch 1 (3/21) // Loss 1.111271 // Accuracy 0.8484
Epoch 1 (4/21) // Loss 1.097109 // Accuracy 0.8547
Epoch 1 (5/21) // Loss 1.085843 // Accuracy 0.8177
Epoch 1 (6/21) // Loss 1.131419 // Accuracy 0.8429
Epoch 1 (7/21) // Loss 1.102834 // Accuracy 0.8203
Epoch 1 (8/21) // Loss 1.141747 // Accuracy 0.8125
Epoch 1 (9/21) // Loss 1.124755 // Accuracy 0.8547
Epoch 1 (10/21) // Loss 1.075223 // Accuracy 0.8368
Epoch 1 (11/21) // Loss 1.121239 // Accuracy 0.8105
Epoch 1 (12/21) // Loss 1.100889 // Accuracy 0.8393
Epoch 1 (13/21) // Loss 1.099409 // Accuracy 0.8277
Epoch 1 (14/21) // Loss 1.116599 // Accuracy 0.8367
Epoch 1 (15/21) // Loss 1.085292 // Accuracy 0.8368
Epoch 1 (16/21) // Loss 1.128865 // Accuracy 0.8217
Epoch 1 (17/21) // Loss 1.108005 // Accuracy 0.8548
Epoch 1 (18/21) // Loss 1.098160 // Accuracy 0.8542
Epoch 1 (19/21) // Loss 1.114966 // Accuracy 0.8165
Epoch 1 (20/21) // Lo

Epoch 8 (13/21) // Loss 1.106129 // Accuracy 0.8504
Epoch 8 (14/21) // Loss 1.069651 // Accuracy 0.8529
Epoch 8 (15/21) // Loss 1.101467 // Accuracy 0.8598
Epoch 8 (16/21) // Loss 1.137871 // Accuracy 0.8029
Epoch 8 (17/21) // Loss 1.093370 // Accuracy 0.8571
Epoch 8 (18/21) // Loss 1.119813 // Accuracy 0.8403
Epoch 8 (19/21) // Loss 1.117108 // Accuracy 0.8333
Epoch 8 (20/21) // Loss 1.125317 // Accuracy 0.8056
Epoch 8 (21/21) // Loss 1.095401 // Accuracy 0.8672
Epoch 9 (1/21) // Loss 1.091237 // Accuracy 0.8735
Epoch 9 (2/21) // Loss 1.085896 // Accuracy 0.8618
Epoch 9 (3/21) // Loss 1.114767 // Accuracy 0.8484
Epoch 9 (4/21) // Loss 1.106500 // Accuracy 0.8455
Epoch 9 (5/21) // Loss 1.125147 // Accuracy 0.8272
Epoch 9 (6/21) // Loss 1.082723 // Accuracy 0.8649
Epoch 9 (7/21) // Loss 1.096720 // Accuracy 0.8590
Epoch 9 (8/21) // Loss 1.093844 // Accuracy 0.8542
Epoch 9 (9/21) // Loss 1.066945 // Accuracy 0.8524
Epoch 9 (10/21) // Loss 1.098126 // Accuracy 0.8185
Epoch 9 (11/21) // Lo

Epoch 16 (2/21) // Loss 1.114645 // Accuracy 0.8013
Epoch 16 (3/21) // Loss 1.120975 // Accuracy 0.7958
Epoch 16 (4/21) // Loss 1.123936 // Accuracy 0.8168
Epoch 16 (5/21) // Loss 1.149055 // Accuracy 0.8203
Epoch 16 (6/21) // Loss 1.098128 // Accuracy 0.8438
Epoch 16 (7/21) // Loss 1.093646 // Accuracy 0.8504
Epoch 16 (8/21) // Loss 1.127571 // Accuracy 0.8271
Epoch 16 (9/21) // Loss 1.141418 // Accuracy 0.8105
Epoch 16 (10/21) // Loss 1.106244 // Accuracy 0.8250
Epoch 16 (11/21) // Loss 1.063730 // Accuracy 0.8504
Epoch 16 (12/21) // Loss 1.081112 // Accuracy 0.8615
Epoch 16 (13/21) // Loss 1.076598 // Accuracy 0.8456
Epoch 16 (14/21) // Loss 1.108287 // Accuracy 0.8409
Epoch 16 (15/21) // Loss 1.109272 // Accuracy 0.8351
Epoch 16 (16/21) // Loss 1.077615 // Accuracy 0.8390
Epoch 16 (17/21) // Loss 1.117730 // Accuracy 0.8304
Epoch 16 (18/21) // Loss 1.139761 // Accuracy 0.8446
Epoch 16 (19/21) // Loss 1.083080 // Accuracy 0.8628
Epoch 16 (20/21) // Loss 1.105765 // Accuracy 0.8320
E

Epoch 23 (11/21) // Loss 1.116894 // Accuracy 0.8208
Epoch 23 (12/21) // Loss 1.094479 // Accuracy 0.8446
Epoch 23 (13/21) // Loss 1.061824 // Accuracy 0.8607
Epoch 23 (14/21) // Loss 1.110555 // Accuracy 0.8352
Epoch 23 (15/21) // Loss 1.107191 // Accuracy 0.8327
Epoch 23 (16/21) // Loss 1.102851 // Accuracy 0.8438
Epoch 23 (17/21) // Loss 1.071323 // Accuracy 0.8576
Epoch 23 (18/21) // Loss 1.094021 // Accuracy 0.8372
Epoch 23 (19/21) // Loss 1.088946 // Accuracy 0.8272
Epoch 23 (20/21) // Loss 1.075102 // Accuracy 0.8574
Epoch 23 (21/21) // Loss 1.114561 // Accuracy 0.8229
Epoch 24 (1/21) // Loss 1.116865 // Accuracy 0.8478
Epoch 24 (2/21) // Loss 1.081444 // Accuracy 0.8598
Epoch 24 (3/21) // Loss 1.072903 // Accuracy 0.8625
Epoch 24 (4/21) // Loss 1.102881 // Accuracy 0.8364
Epoch 24 (5/21) // Loss 1.087212 // Accuracy 0.8603
Epoch 24 (6/21) // Loss 1.106595 // Accuracy 0.8214
Epoch 24 (7/21) // Loss 1.133060 // Accuracy 0.8391
Epoch 24 (8/21) // Loss 1.116987 // Accuracy 0.8083
E

Epoch 30 (20/21) // Loss 1.112801 // Accuracy 0.8314
Epoch 30 (21/21) // Loss 1.104452 // Accuracy 0.8229
Epoch 31 (1/21) // Loss 1.076699 // Accuracy 0.8664
Epoch 31 (2/21) // Loss 1.127024 // Accuracy 0.8262
Epoch 31 (3/21) // Loss 1.085803 // Accuracy 0.8223
Epoch 31 (4/21) // Loss 1.068030 // Accuracy 0.8421
Epoch 31 (5/21) // Loss 1.092383 // Accuracy 0.8418
Epoch 31 (6/21) // Loss 1.116153 // Accuracy 0.8438
Epoch 31 (7/21) // Loss 1.121437 // Accuracy 0.8235
Epoch 31 (8/21) // Loss 1.141210 // Accuracy 0.8235
Epoch 31 (9/21) // Loss 1.093094 // Accuracy 0.8526
Epoch 31 (10/21) // Loss 1.153839 // Accuracy 0.7775
Epoch 31 (11/21) // Loss 1.108707 // Accuracy 0.8382
Epoch 31 (12/21) // Loss 1.107161 // Accuracy 0.8286
Epoch 31 (13/21) // Loss 1.074938 // Accuracy 0.8598
Epoch 31 (14/21) // Loss 1.124524 // Accuracy 0.8390
Epoch 31 (15/21) // Loss 1.065887 // Accuracy 0.8529
Epoch 31 (16/21) // Loss 1.122717 // Accuracy 0.8292
Epoch 31 (17/21) // Loss 1.073172 // Accuracy 0.8598
Ep

...

Epoch 921 (3/21) // Loss 1.088595 // Accuracy 0.8526
Epoch 921 (4/21) // Loss 1.084824 // Accuracy 0.8652
Epoch 921 (5/21) // Loss 1.113015 // Accuracy 0.8056
Epoch 921 (6/21) // Loss 1.082646 // Accuracy 0.8226
Epoch 921 (7/21) // Loss 1.101021 // Accuracy 0.8354
Epoch 921 (8/21) // Loss 1.112731 // Accuracy 0.8313
Epoch 921 (9/21) // Loss 1.074840 // Accuracy 0.8602
Epoch 921 (10/21) // Loss 1.104931 // Accuracy 0.8585
Epoch 921 (11/21) // Loss 1.061364 // Accuracy 0.8681
Epoch 921 (12/21) // Loss 1.078223 // Accuracy 0.8553
Epoch 921 (13/21) // Loss 1.088281 // Accuracy 0.8536
Epoch 921 (14/21) // Loss 1.105913 // Accuracy 0.8457
Epoch 921 (15/21) // Loss 1.117180 // Accuracy 0.8190
Epoch 921 (16/21) // Loss 1.065804 // Accuracy 0.8625
Epoch 921 (17/21) // Loss 1.072230 // Accuracy 0.8520
Epoch 921 (18/21) // Loss 1.077497 // Accuracy 0.8474
Epoch 921 (19/21) // Loss 1.100967 // Accuracy 0.8125
Epoch 921 (20/21) // Loss 1.072916 // Accuracy 0.8523
Epoch 921 (21/21) // Loss 1.090556 

Epoch 928 (9/21) // Loss 1.083129 // Accuracy 0.8438
Epoch 928 (10/21) // Loss 1.094223 // Accuracy 0.8523
Epoch 928 (11/21) // Loss 1.110884 // Accuracy 0.8290
Epoch 928 (12/21) // Loss 1.072906 // Accuracy 0.8548
Epoch 928 (13/21) // Loss 1.088184 // Accuracy 0.8393
Epoch 928 (14/21) // Loss 1.084169 // Accuracy 0.8504
Epoch 928 (15/21) // Loss 1.123018 // Accuracy 0.8429
Epoch 928 (16/21) // Loss 1.100400 // Accuracy 0.8670
Epoch 928 (17/21) // Loss 1.100528 // Accuracy 0.8346
Epoch 928 (18/21) // Loss 1.121451 // Accuracy 0.8281
Epoch 928 (19/21) // Loss 1.114270 // Accuracy 0.8396
Epoch 928 (20/21) // Loss 1.077367 // Accuracy 0.8566
Epoch 928 (21/21) // Loss 1.137909 // Accuracy 0.7984
Epoch 929 (1/21) // Loss 1.082542 // Accuracy 0.8554
Epoch 929 (2/21) // Loss 1.097817 // Accuracy 0.8559
Epoch 929 (3/21) // Loss 1.079283 // Accuracy 0.8487
Epoch 929 (4/21) // Loss 1.119999 // Accuracy 0.8301
Epoch 929 (5/21) // Loss 1.094966 // Accuracy 0.8413
Epoch 929 (6/21) // Loss 1.082816 

Epoch 935 (15/21) // Loss 1.078756 // Accuracy 0.8413
Epoch 935 (16/21) // Loss 1.130452 // Accuracy 0.8149
Epoch 935 (17/21) // Loss 1.081627 // Accuracy 0.8651
Epoch 935 (18/21) // Loss 1.110315 // Accuracy 0.8405
Epoch 935 (19/21) // Loss 1.074010 // Accuracy 0.8306
Epoch 935 (20/21) // Loss 1.087585 // Accuracy 0.8464
Epoch 935 (21/21) // Loss 1.128579 // Accuracy 0.8313
Epoch 936 (1/21) // Loss 1.090832 // Accuracy 0.8684
Epoch 936 (2/21) // Loss 1.108484 // Accuracy 0.8536
Epoch 936 (3/21) // Loss 1.133583 // Accuracy 0.8403
Epoch 936 (4/21) // Loss 1.047894 // Accuracy 0.8821
Epoch 936 (5/21) // Loss 1.098483 // Accuracy 0.8493
Epoch 936 (6/21) // Loss 1.088094 // Accuracy 0.8346
Epoch 936 (7/21) // Loss 1.136831 // Accuracy 0.8409
Epoch 936 (8/21) // Loss 1.097389 // Accuracy 0.8594
Epoch 936 (9/21) // Loss 1.073282 // Accuracy 0.8472
Epoch 936 (10/21) // Loss 1.096461 // Accuracy 0.8542
Epoch 936 (11/21) // Loss 1.089112 // Accuracy 0.8698
Epoch 936 (12/21) // Loss 1.076560 //

Epoch 942 (21/21) // Loss 1.097852 // Accuracy 0.8339
Epoch 943 (1/21) // Loss 1.086974 // Accuracy 0.8438
Epoch 943 (2/21) // Loss 1.093583 // Accuracy 0.8793
Epoch 943 (3/21) // Loss 1.100844 // Accuracy 0.8456
Epoch 943 (4/21) // Loss 1.080367 // Accuracy 0.8590
Epoch 943 (5/21) // Loss 1.080891 // Accuracy 0.8676
Epoch 943 (6/21) // Loss 1.074404 // Accuracy 0.8339
Epoch 943 (7/21) // Loss 1.147143 // Accuracy 0.8321
Epoch 943 (8/21) // Loss 1.072225 // Accuracy 0.8542
Epoch 943 (9/21) // Loss 1.072227 // Accuracy 0.8646
Epoch 943 (10/21) // Loss 1.111889 // Accuracy 0.8316
Epoch 943 (11/21) // Loss 1.066112 // Accuracy 0.8576
Epoch 943 (12/21) // Loss 1.094384 // Accuracy 0.8418
Epoch 943 (13/21) // Loss 1.112520 // Accuracy 0.8472
Epoch 943 (14/21) // Loss 1.113879 // Accuracy 0.8559
Epoch 943 (15/21) // Loss 1.047382 // Accuracy 0.8589
Epoch 943 (16/21) // Loss 1.114465 // Accuracy 0.8382
Epoch 943 (17/21) // Loss 1.094350 // Accuracy 0.8536
Epoch 943 (18/21) // Loss 1.090823 //

Epoch 950 (6/21) // Loss 1.092549 // Accuracy 0.8403
Epoch 950 (7/21) // Loss 1.068924 // Accuracy 0.8555
Epoch 950 (8/21) // Loss 1.059083 // Accuracy 0.8782
Epoch 950 (9/21) // Loss 1.074867 // Accuracy 0.8500
Epoch 950 (10/21) // Loss 1.101789 // Accuracy 0.8164
Epoch 950 (11/21) // Loss 1.091228 // Accuracy 0.8299
Epoch 950 (12/21) // Loss 1.091103 // Accuracy 0.8464
Epoch 950 (13/21) // Loss 1.073735 // Accuracy 0.8258
Epoch 950 (14/21) // Loss 1.091900 // Accuracy 0.8507
Epoch 950 (15/21) // Loss 1.090357 // Accuracy 0.8602
Epoch 950 (16/21) // Loss 1.064239 // Accuracy 0.8679
Epoch 950 (17/21) // Loss 1.072542 // Accuracy 0.8507
Epoch 950 (18/21) // Loss 1.095625 // Accuracy 0.8333
Epoch 950 (19/21) // Loss 1.162339 // Accuracy 0.8188
Epoch 950 (20/21) // Loss 1.059812 // Accuracy 0.8520
Epoch 950 (21/21) // Loss 1.066289 // Accuracy 0.8684
Epoch 951 (1/21) // Loss 1.080693 // Accuracy 0.8451
Epoch 951 (2/21) // Loss 1.073479 // Accuracy 0.8423
Epoch 951 (3/21) // Loss 1.115571 